## INFOSYS 722 Assignement Iteration 4

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
spark = SparkSession.builder.appName('infosys722-i4').getOrCreate()

### Read Raw Data

In [3]:
raw_data_products = spark.read.options(compression='gzip').csv('./Datasets/2017PurchasePricesDec.csv.gz', inferSchema=True, header=True)
raw_data_inventory = spark.read.options(compression='gzip').csv('./Datasets/BegInvFINAL12312016.csv.gz', inferSchema=True, header=True)
raw_data_inventory_end = spark.read.options(compression='gzip').csv('./Datasets/EndInvFINAL12312016.csv.gz', inferSchema=True, header=True)
raw_data_purchase_order = spark.read.options(compression='gzip').csv('./Datasets/InvoicePurchases12312016.csv.gz', inferSchema=True, header=True)
raw_data_purchase_detail = spark.read.options(compression='gzip').csv('./Datasets/PurchasesFINAL12312016.csv.gz', inferSchema=True, header=True)
raw_data_sales = spark.read.options(compression='gzip').csv('./Datasets/SalesFINAL12312016.csv.gz', inferSchema=True, header=True)

### Data Understanding

#### Describe Data

In [4]:
raw_data_products.head()

Row(Brand=58, Description='Gekkeikan Black & Gold Sake', Price=12.99, Size='750mL', Volume='750', Classification=1, PurchasePrice=9.28, VendorNumber=8320, VendorName='SHAW ROSS INT L IMP LTD    ')

In [5]:
raw_data_inventory.head()

Row(InventoryId='1_HARDERSFIELD_58', Store=1, City='HARDERSFIELD', Brand=58, Description='Gekkeikan Black & Gold Sake', Size='750mL', onHand=8, Price=12.99, startDate=datetime.date(2016, 1, 1))

In [6]:
raw_data_inventory_end.head()

Row(InventoryId='1_HARDERSFIELD_58', Store=1, City='HARDERSFIELD', Brand=58, Description='Gekkeikan Black & Gold Sake', Size='750mL', onHand=11, Price=12.99, endDate=datetime.date(2016, 12, 31))

In [7]:
raw_data_purchase_order.head()

Row(VendorNumber=105, VendorName='ALTAMAR BRANDS LLC         ', InvoiceDate=datetime.date(2016, 1, 4), PONumber=8124, PODate=datetime.date(2015, 12, 21), PayDate=datetime.date(2016, 2, 16), Quantity=6, Dollars=214.26, Freight=3.47, Approval='None')

In [8]:
raw_data_purchase_detail.head()

Row(InventoryId='69_MOUNTMEND_8412', Store=69, Brand=8412, Description='Tequila Ocho Plata Fresno', Size='750mL', VendorNumber=105, VendorName='ALTAMAR BRANDS LLC         ', PONumber=8124, PODate=datetime.date(2015, 12, 21), ReceivingDate=datetime.date(2016, 1, 2), InvoiceDate=datetime.date(2016, 1, 4), PayDate=datetime.date(2016, 2, 16), PurchasePrice=35.71, Quantity=6, Dollars=214.26, Classification=1)

In [9]:
raw_data_sales.head()

Row(InventoryId='1_HARDERSFIELD_1004', Store=1, Brand=1004, Description='Jim Beam w/2 Rocks Glasses', Size='750mL', SalesQuantity=1, SalesDollars=16.49, SalesPrice=16.49, SalesDate='1/1/2016', Volume=750, Classification=1, ExciseTax=0.79, VendorNo=12546, VendorName='JIM BEAM BRANDS COMPANY    ')

In [10]:
def print_summary(dataframe):
    summary = dataframe.summary()
    columns = ['feature'] + [row['summary'] for row in summary.collect()]
    data = []
    for feature in summary.columns[1:]:
        data.append([feature, *[row[feature] for row in summary.collect()]])
    spark.createDataFrame(data, schema=columns).show()

In [11]:
raw_data_products.printSchema()
print_summary(raw_data_products)

root
 |-- Brand: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Size: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Classification: integer (nullable = true)
 |-- PurchasePrice: double (nullable = true)
 |-- VendorNumber: integer (nullable = true)
 |-- VendorName: string (nullable = true)

+--------------+-----+------------------+------------------+--------------------+-----+-----+-----+--------------------+
|       feature|count|              mean|            stddev|                 min|  25%|  50%|  75%|                 max|
+--------------+-----+------------------+------------------+--------------------+-----+-----+-----+--------------------+
|         Brand|12261|17989.067123399396|12528.503463986654|                  58| 5989|18787|25113|               90631|
|   Description|12260|              null|              null|              (RI) 1| null| null| null|   von Buhl Jazz Rsl|
|         Price|12

In [12]:
raw_data_inventory.printSchema()
print_summary(raw_data_inventory)

root
 |-- InventoryId: string (nullable = true)
 |-- Store: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Brand: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- onHand: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- startDate: date (nullable = true)

+-----------+------+------------------+------------------+--------------------+----+-----+-----+-----------------+
|    feature| count|              mean|            stddev|                 min| 25%|  50%|  75%|              max|
+-----------+------+------------------+------------------+--------------------+----+-----+-----+-----------------+
|InventoryId|206529|              null|              null|     10_HORNSEY_1000|null| null| null|  9_BLACKPOOL_999|
|      Store|206529| 42.12245737886689|23.191393409648104|                   1|  22|   42|   64|               79|
|       City|206529|              null|              null|            

In [13]:
raw_data_inventory_end.printSchema()
print_summary(raw_data_inventory_end)

root
 |-- InventoryId: string (nullable = true)
 |-- Store: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Brand: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- onHand: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- endDate: date (nullable = true)

+-----------+------+------------------+------------------+--------------------+----+-----+-----+-----------------+
|    feature| count|              mean|            stddev|                 min| 25%|  50%|  75%|              max|
+-----------+------+------------------+------------------+--------------------+----+-----+-----+-----------------+
|InventoryId|224489|              null|              null|     10_HORNSEY_1001|null| null| null|  9_BLACKPOOL_984|
|      Store|224489|43.505739702168036|23.326415082417782|                   1|  23|   44|   66|               81|
|       City|223205|              null|              null|            AB

In [14]:
raw_data_purchase_order.printSchema()
print_summary(raw_data_purchase_order)

root
 |-- VendorNumber: integer (nullable = true)
 |-- VendorName: string (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- PONumber: integer (nullable = true)
 |-- PODate: date (nullable = true)
 |-- PayDate: date (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Dollars: double (nullable = true)
 |-- Freight: double (nullable = true)
 |-- Approval: string (nullable = true)

+------------+-----+------------------+------------------+--------------------+------+-------+--------+--------------------+
|     feature|count|              mean|            stddev|                 min|   25%|    50%|     75%|                 max|
+------------+-----+------------------+------------------+--------------------+------+-------+--------+--------------------+
|VendorNumber| 5543|20662.752119790726| 34582.15840981278|                   2|  3089|   7240|   10754|              201359|
|  VendorName| 5543|              null|              null|AAPER ALCOHOL & C...|  null|   null

In [15]:
raw_data_purchase_detail.printSchema()
print_summary(raw_data_purchase_detail)

root
 |-- InventoryId: string (nullable = true)
 |-- Store: integer (nullable = true)
 |-- Brand: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- VendorNumber: integer (nullable = true)
 |-- VendorName: string (nullable = true)
 |-- PONumber: integer (nullable = true)
 |-- PODate: date (nullable = true)
 |-- ReceivingDate: date (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- PayDate: date (nullable = true)
 |-- PurchasePrice: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Dollars: double (nullable = true)
 |-- Classification: integer (nullable = true)

+--------------+-------+------------------+------------------+--------------------+-----+-----+------+--------------------+
|       feature|  count|              mean|            stddev|                 min|  25%|  50%|   75%|                 max|
+--------------+-------+------------------+------------------+--------------------+-----+--

In [17]:
raw_data_sales.printSchema()
print_summary(raw_data_sales)

root
 |-- InventoryId: string (nullable = true)
 |-- Store: integer (nullable = true)
 |-- Brand: integer (nullable = true)
 |-- Description: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- SalesQuantity: integer (nullable = true)
 |-- SalesDollars: double (nullable = true)
 |-- SalesPrice: double (nullable = true)
 |-- SalesDate: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Classification: integer (nullable = true)
 |-- ExciseTax: double (nullable = true)
 |-- VendorNo: integer (nullable = true)
 |-- VendorName: string (nullable = true)

+--------------+-------+------------------+-------------------+--------------------+-----+-----+-----+--------------------+
|       feature|  count|              mean|             stddev|                 min|  25%|  50%|  75%|                 max|
+--------------+-------+------------------+-------------------+--------------------+-----+-----+-----+--------------------+
|   InventoryId|1048575|              nul

In [19]:
def replace(dataframe, column, condition, new_value):
    return dataframe.withColumn(column, F.when(condition, new_value).otherwise(dataframe[column]))

#
# replace invalid value
#
raw_data_products = replace(raw_data_products, 'Volume', raw_data_products['Volume'].isNull(), 0)
raw_data_products = replace(raw_data_products, 'Volume', raw_data_products['Volume'] == 'Unknown', 0)
raw_data_products = replace(raw_data_products, 'Volume', raw_data_products['Volume'] == '162.5', 162)

#
# Correct Data Type
#
dtype_products = {
    'Brand': 'string',
    'Description': 'string',
    'Price': 'float',
    'Size': 'string',
    'Volume': 'int',
    'Classification': 'string',
    'PurchasePrice': 'float',
    'VendorNumber': 'string',
    'VendorName': 'string',
}
    
dtype_inventory_begin = {    
    'InventoryId': 'string',
    'Store': 'string',
    'City': 'string',
    'Brand': 'string',
    'Description': 'string',
    'Size': 'string',
    'onHand': 'int',
    'Price': 'float',
    'startDate': 'string',
}    

dtype_inventory_end = { 
    'InventoryId': 'string',
    'Store': 'string',
    'City': 'string',
    'Brand': 'string',
    'Description': 'string',
    'Size': 'string',
    'onHand': 'int',
    'Price': 'float',
    'endDate': 'string',
}    

dtype_purchase_order = {    
    'VendorNumber': 'string',
    'VendorName': 'string',
    'InvoiceDate': 'string',
    'PONumber': 'string',
    'PODate': 'string',
    'PayDate': 'string',
    'Quantity': 'int',
    'Dollars': 'float',
    'Freight': 'float',
    'Approval': 'string',
}

dtype_purchase_detail = {
    'InventoryId': 'string',
    'Store': 'string',
    'Brand': 'string',
    'Description': 'string',
    'Size': 'string',
    'VendorNumber': 'string',
    'VendorName': 'string',
    'PONumber': 'string',
    'PODate': 'string',
    'ReceivingDate': 'string',
    'InvoiceDate': 'string',
    'PayDate': 'string',
    'PurchasePrice': 'float',
    'Quantity': 'int',
    'Dollars': 'float',
    'Classification': 'string',
}   
    
dtype_sales = {    
    'InventoryId': 'string',
    'Store': 'string',
    'Brand': 'string',
    'Description': 'string',
    'Size': 'string',
    'SalesQuantity': 'int',
    'SalesDollars': 'float',
    'SalesPrice': 'float',
    'SalesDate': 'string',
    'Volume': 'int',
    'Classification': 'string',
    'ExciseTax': 'float',
    'VendorNo': 'string',
    'VendorName': 'string',
}

def correct_dtype(dataframe, dtype_dict):
    for column, dtype in dtype_dict.items():
        dataframe = dataframe.withColumn(column, F.col(column).cast(dtype))
    return dataframe

raw_data_products = correct_dtype(raw_data_products, dtype_products)
raw_data_inventory = correct_dtype(raw_data_inventory, dtype_inventory_begin)
raw_data_inventory_end = correct_dtype(raw_data_inventory_end, dtype_inventory_end)
raw_data_purchase_order = correct_dtype(raw_data_purchase_order, dtype_purchase_order)
raw_data_purchase_detail = correct_dtype(raw_data_purchase_detail, dtype_purchase_detail)
raw_data_sales = correct_dtype(raw_data_sales, dtype_sales)

#
# Convert date string to datetime
#

def format_date(dataframe, columns, date_format='yyyy-MM-dd'):
    for column in columns:
        dataframe = dataframe.withColumn(column, F.to_date(column, date_format))
    return dataframe
        
raw_data_inventory = format_date(raw_data_inventory, ['startDate'])
raw_data_inventory_end = format_date(raw_data_inventory_end, ['endDate'])
raw_data_purchase_order = format_date(raw_data_purchase_order, ['PODate', 'InvoiceDate', 'PayDate'])
raw_data_purchase_detail = format_date(raw_data_purchase_detail, ['PODate', 'ReceivingDate', 'InvoiceDate', 'PayDate'])
raw_data_sales = format_date(raw_data_sales, ['SalesDate'], date_format = 'M/d/yyyy')

#
# Align features' name across datasets
#
# VendorNo in Sales should be renamed to VendorNumber
raw_data_sales = raw_data_sales.withColumnRenamed('VendorNo', 'VendorNumber')

In [20]:
raw_data_products.printSchema()

root
 |-- Brand: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- Size: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Classification: string (nullable = true)
 |-- PurchasePrice: float (nullable = true)
 |-- VendorNumber: string (nullable = true)
 |-- VendorName: string (nullable = true)



In [21]:
raw_data_inventory.printSchema()

root
 |-- InventoryId: string (nullable = true)
 |-- Store: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- onHand: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- startDate: date (nullable = true)



In [22]:
raw_data_inventory_end.printSchema()

root
 |-- InventoryId: string (nullable = true)
 |-- Store: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- onHand: integer (nullable = true)
 |-- Price: float (nullable = true)
 |-- endDate: date (nullable = true)



In [26]:
raw_data_purchase_order.printSchema()

root
 |-- VendorNumber: string (nullable = true)
 |-- VendorName: string (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- PONumber: string (nullable = true)
 |-- PODate: date (nullable = true)
 |-- PayDate: date (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Dollars: float (nullable = true)
 |-- Freight: float (nullable = true)
 |-- Approval: string (nullable = true)



In [27]:
raw_data_purchase_detail.printSchema()

root
 |-- InventoryId: string (nullable = true)
 |-- Store: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- VendorNumber: string (nullable = true)
 |-- VendorName: string (nullable = true)
 |-- PONumber: string (nullable = true)
 |-- PODate: date (nullable = true)
 |-- ReceivingDate: date (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- PayDate: date (nullable = true)
 |-- PurchasePrice: float (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Dollars: float (nullable = true)
 |-- Classification: string (nullable = true)



In [28]:
raw_data_sales.printSchema()

root
 |-- InventoryId: string (nullable = true)
 |-- Store: string (nullable = true)
 |-- Brand: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- SalesQuantity: integer (nullable = true)
 |-- SalesDollars: float (nullable = true)
 |-- SalesPrice: float (nullable = true)
 |-- SalesDate: date (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Classification: string (nullable = true)
 |-- ExciseTax: float (nullable = true)
 |-- VendorNumber: string (nullable = true)
 |-- VendorName: string (nullable = true)



In [31]:
selected_inventory = raw_data_inventory.where('Store = 15')
selected_purchase_detail = raw_data_purchase_detail.where('Store = 15')
selected_sales = raw_data_sales.where('Store = 15')

In [37]:
selected_inventory.select('Store').distinct().collect()

[Row(Store='15')]

In [38]:
selected_purchase_detail.select('Store').distinct().collect()

[Row(Store='15')]

In [39]:
selected_sales.select('Store').distinct().collect()

[Row(Store='15')]